In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import utils
from tensorflow.keras import preprocessing

/Users/humanperson/anaconda3/envs/learn2env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [3]:
train = pd.read_csv('data/train.csv')
train.head()

,id,label
0,1,9
1,2,0
2,3,0
3,4,3
4,5,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 2 columns):
id       60000 non-null int64
label    60000 non-null int64
dtypes: int64(2)
memory usage: 937.6 KB


In [5]:
# load our grayscale images
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('data/train/'+train['id'][i].astype('str')+'.png', target_size=(28,28,1), color_mode = "grayscale")
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 60000/60000 [00:32<00:00, 1837.52it/s]


## Data preparation

In [6]:
y=train['label'].values
y = to_categorical(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

## Define our model structure

In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


## Compile our model

In [9]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [10]:
# Training the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 53s 1ms/step - loss: 0.3908 - acc: 0.8602 - val_loss: 0.3002 - val_acc: 0.8871
Epoch 2/10
48000/48000 [==============================] - 56s 1ms/step - loss: 0.2451 - acc: 0.9113 - val_loss: 0.2483 - val_acc: 0.9098
Epoch 3/10
48000/48000 [==============================] - 52s 1ms/step - loss: 0.1850 - acc: 0.9320 - val_loss: 0.2598 - val_acc: 0.9073
Epoch 4/10
48000/48000 [==============================] - 51s 1ms/step - loss: 0.1446 - acc: 0.9471 - val_loss: 0.2433 - val_acc: 0.9154
Epoch 5/10
48000/48000 [==============================] - 51s 1ms/step - loss: 0.1072 - acc: 0.9600 - val_loss: 0.2595 - val_acc: 0.9162
Epoch 6/10
48000/48000 [==============================] - 51s 1ms/step - loss: 0.0782 - acc: 0.9703 - val_loss: 0.2610 - val_acc: 0.9243
Epoch 7/10
48000/48000 [==============================] - 53s 1ms/step - loss: 0

##  After 10 epochs the accuracy is given as 0.9225, which seems good. Let's persist our model.

In [14]:
import pickle

In [12]:
filename = 'clothes_model0.pkl'
pickle.dump(model, open(filename, 'wb'))

# Use the model to populate unlabeled image data for submission to https://datahack.analyticsvidhya.com/contest/practice-problem-identify-the-apparels/#solution_checker

In [15]:
with open('clothes_model0.pkl', 'rb') as f:
    model = pickle.load(f)

## Bring in data

In [16]:
dftest = pd.read_csv('data/test.csv')
dftest.head()

,id
0,60001
1,60002
2,60003
3,60004
4,60005


In [18]:
test_image = []
for i in tqdm(range(dftest.shape[0])):
    img = image.load_img('data/test/'+dftest['id'][i].astype('str')+'.png', target_size=(28,28,1), color_mode='grayscale')
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

100%|██████████| 10000/10000 [00:02<00:00, 4187.91it/s]


## Make our predictions

In [ ]:
predictions = model.predict_classes(test)

In [ ]:
df_predictions = dftest.copy()
df_predictions['prediction'] = predictions
df_predictions.head()

## Output our predictions to a for submission to https://datahack.analyticsvidhya.com/contest/practice-problem-identify-the-apparels/#solution_checker

In [ ]:
sample = pd.read_csv('data/sample_submission_I5njJSF (1).csv')
sample['id'] = df_predictions['id']
sample['label'] = df_predictions['prediction']
sample.to_csv('sample.csv', header=True, index=False)